In [32]:
from IPython.display import display
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from enum import Enum
from copy import deepcopy
from pprint import pprint
from functools import reduce, cmp_to_key
from itertools import groupby, combinations, permutations
from time import time
from queue import PriorityQueue
from math import prod, ceil, comb, perm
from collections import defaultdict
from dataclasses import dataclass
from bisect import insort
import string

files = [f for f in listdir('.') if isfile(join('.', f))]
pat = re.compile('(ex(\d)+.txt)|(in(\d)*.txt)')
files = [f for f in files if pat.match(f)]

box = widgets.Select(
    options=files,
    value='ex1.txt',
    rows=4,
    description='Input file:',
    disabled=False
)

display(box)


Select(description='Input file:', index=1, options=('in.txt', 'ex1.txt'), rows=4, value='ex1.txt')

## Parse file

In [37]:
lines = list(map(lambda l: l.rstrip(), open(box.value, 'r').readlines()))

X = np.array(list(map(int, lines)))

print("Done parsing \"{name}\"".format(name=box.value))


Done parsing "in.txt"


### Part 1

In [38]:
def insert(C, i, l, r):
    C[l][1] = i
    C[r][0] = i
    C[i] = [l, r]


def join(C, l, r):
    C[l][1] = r
    C[r][0] = l


def decrypt(E, iter, KEY):
    C = {}
    n = len(E)
    X = list(map(lambda x: x * KEY, E))

    for i in range(n):
        C[i] = [(i-1) % n, (i+1) % n]

    for k in range(iter):
        for i in range(n):
            x = X[i] % (n-1)
            if x == 0:
                continue

            l, r = C[i]
            join(C, l, r)
            j = i
            left = x < 0
            while x != 0:
                if x < 0:
                    j = C[j][0]
                    x += 1
                else:
                    j = C[j][1]
                    x -= 1

            if left:
                insert(C, i, C[j][0], j)
            else:
                insert(C, i, j, C[j][1])

    i = np.where(E == 0)[0][0]
    R = [X[i]]
    for _ in range(n-1):
        i = C[i][1]
        R.append(X[i])

    return sum([R[i % n] for i in [1000, 2000, 3000]])


In [39]:
print(decrypt(X, 1, 1))


7153


### Part 2

In [40]:
KEY = 811589153
print(decrypt(X, 10, KEY))


6146976244822
